*Please notebook is written to run on Google Colab.*

# Install detectron2

In [ ]:
# install dependencies: 
!pip install pyyaml==5.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

In [ ]:
# install detectron2: (Colab has CUDA 10.1 + torch 1.7)
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
import torch
assert torch.__version__.startswith("1.7")
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.7/index.html
exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime

# Import libraries

In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog, DatasetCatalog, build_detection_train_loader, DatasetMapper
import detectron2.data.transforms as T

# Connect G Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Paths

In [ ]:
PROJECT_ROOT=''
OUTPUT_DIR=os.path.join(PROJECT_ROOT, 'output')

# Prepare dataset

In [ ]:
from detectron2.data.datasets import register_coco_instances
for d in ["train", "val"]:
    register_coco_instances(
        "coco_" + d,
        {},
        os.path.join(PROJECT_ROOT, "coco_" + d, "annotations.json"),
        os.path.join(PROJECT_ROOT, "coco_" + d),
    )

# Model Configuration

In [ ]:
def setup():
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
    cfg.DATASETS.TRAIN = ("coco_train",)
    cfg.DATASETS.TEST = ()
    cfg.DATALOADER.NUM_WORKERS = 2
    cfg.INPUT.MIN_SIZE_TRAIN = 512
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
    cfg.SOLVER.IMS_PER_BATCH = 1
    cfg.SOLVER.BASE_LR = 0.00025
    cfg.SOLVER.MAX_ITER = 3000
    cfg.OUTPUT_DIR = OUTPUT_DIR
    if torch.cuda.is_available():
        cfg.MODEL.DEVICE = "cuda"
    else:
        cfg.MODEL.DEVICE = "cpu"
    return cfg

# Data Augmentation

In [1]:
def train_aug(cfg):
    augs = [
        T.ResizeShortestEdge(short_edge_length=(512, 512), max_size=1333, sample_style='choice'),
        T.RandomFlip(),
        T.RandomBrightness(intensity_min=0.6, intensity_max=1.2),
    ]
    return augs

# Trainer


In [ ]:
class Trainer(DefaultTrainer):
    @classmethod
    def build_train_loader(cls, cfg):
        mapper = DatasetMapper(cfg, is_train=True, augmentations=train_aug(cfg))
        return build_detection_train_loader(cfg=cfg, mapper=mapper)

# Train model

In [ ]:
cfg = setup()
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = Trainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

# Predictor

# Evaluate

In [ ]:
cfg = setup()
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9
predictor = DefaultPredictor(cfg)

In [ ]:
from google.colab import files

uploaded = files.upload()

In [ ]:
for fn in uploaded.keys():
    nparr = np.frombuffer(uploaded[fn],'u1')    
    im = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    plot1 = plt.figure(1)
    plt.imshow(im)
    outputs = predictor(im)
    mask = np.asarray(outputs['instances'].pred_masks.cpu().numpy()[0], dtype=np.uint8)
    plot2 = plt.figure(2)
    plt.imshow(mask)
    cropped = cv2.bitwise_and(im, im, mask=mask)
    plot3 = plt.figure(3)
    plt.imshow(cropped)
    plt.show()

## AP Score

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("coco_val", cfg, False, output_dir=output_dir)
val_loader = build_detection_test_loader(cfg, "coco_val")
print(inference_on_dataset(trainer.model, val_loader, evaluator))
# another equivalent way to evaluate the model is to use `trainer.test`